<a href="https://colab.research.google.com/github/NeuralClassifier/ReusabilityRate_ML/blob/main/CMPUT_663_ReuseRate_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statistics

import warnings

In [2]:
!git clone https://github.com/NeuralClassifier/ReusabilityRate_ML

Cloning into 'ReusabilityRate_ML'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 83 (delta 29), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [28]:
df = pd.read_excel('/content/ReusabilityRate_ML/DataSets/packages.xlsx')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df

,Project,LongName,Name,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL,ReuseRate
0,ch.qos.logback/logback-classic,<root_package>,<root_package>,0.000000,0.000000,0.342362,0,4146,0,219,0,0,14288,0,7964,403,217,853,3498,624,0,0
1,ch.qos.logback/logback-classic,ch,ch,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
2,ch.qos.logback/logback-classic,unnamed package,unnamed package,0.000000,0.000000,0.000000,0,0,0,0,32,38,38,0,32,0,0,0,0,0,0,0
3,ch.qos.logback/logback-classic,ch.qos,qos,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
4,ch.qos.logback/logback-classic,ch.qos.logback,logback,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,xerces/xercesImpl,org.apache.xerces.impl.xs.identity,identity,0.984127,0.498253,0.498253,713,713,62,62,718,1603,1603,22,718,39,22,66,394,53,10,0
2189,xerces/xercesImpl,org.apache.xerces.impl.xs.models,models,0.701493,0.476530,0.476530,1137,1137,47,47,1249,2719,2719,17,1249,52,17,91,732,59,10,0
2190,xerces/xercesImpl,org.apache.xerces.impl.xs.opti,opti,0.540636,0.482071,0.482071,1909,1909,153,153,2051,4621,4621,94,2051,96,94,312,851,280,15,3
2191,xerces/xercesImpl,org.apache.xerces.impl.xs.traversers,traversers,0.700000,0.225403,0.225403,2543,2543,14,14,8739,12621,12621,34,8739,332,34,290,4838,57,29,0


In [33]:
df.iloc[:,3:-1]

,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL
0,0.000000,0.000000,0.342362,0,4146,0,219,0,0,14288,0,7964,403,217,853,3498,624,0
1,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
2,0.000000,0.000000,0.000000,0,0,0,0,32,38,38,0,32,0,0,0,0,0,0
3,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
4,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,0.984127,0.498253,0.498253,713,713,62,62,718,1603,1603,22,718,39,22,66,394,53,10
2189,0.701493,0.476530,0.476530,1137,1137,47,47,1249,2719,2719,17,1249,52,17,91,732,59,10
2190,0.540636,0.482071,0.482071,1909,1909,153,153,2051,4621,4621,94,2051,96,94,312,851,280,15
2191,0.700000,0.225403,0.225403,2543,2543,14,14,8739,12621,12621,34,8739,332,34,290,4838,57,29


In [30]:
len(rtm.X_np[:,0])

24794

In [46]:
warnings.filterwarnings('ignore')
class ReuseTradML:
  def __init__(self,level):
    self.ml_algo_list = ['PolR',
                         'MLPR',
                         'BR',
                         'SGDR',
                         'XGBR',
                         'DTR',
                         'EN',
                         'SVR']

    self.metrics = ['RMSE',
                    'MAE']
    self.data_columns = None
    self.level=level

    if self.level == 'package':
      target_scaler = MinMaxScaler()
      data = pd.read_excel('/content/ReusabilityRate_ML/DataSets/packages.xlsx')
      data.drop(['Unnamed: 0'],axis=1,inplace=True)
      df = data.iloc[:,3:-1]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      #target_scaler = MinMaxScaler()
      #target_scaler.fit(self.X_np)
      #self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'class':
      target_scaler = MinMaxScaler()
      data = pd.read_excel('/content/ReusabilityRate_ML/DataSets/classes.xlsx')
      data.drop(['Unnamed: 0'],axis=1,inplace=True)
      df = data.iloc[:,3:-1]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      #target_scaler = MinMaxScaler()
      #target_scaler.fit(self.X_np)
      #self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'project':
      self.X_np = np.array(data.iloc[:,4:])
      self.other = data.iloc[:,:4]

    else:
      print('Error, Please try: class, package, or project. Thanks!')
      self.X=None
      self.Y=None
      self.other=None

  def apply_ml(self):
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
  
    metric = []
    metric_count = 0

    level = []
    model = []

    mean_RMSE = []
    stdv_RMSE = []
    mean_MAE = []
    stdv_MAE = []

    for i in range(self.X_np.shape[1]):
      #print(self.X[i])
      for j in self.ml_algo_list:
        if j == 'PolR':
          
          #temp_data = self.X[i]
          level.append(self.level)
          metric.append(self.data_columns[i])

          cnt=0
          RMSE=[]
          MAE=[]
          poly_reg = PolynomialFeatures(degree=7)
          temp_data = poly_reg.fit_transform(self.X_np[:,i].reshape(-1, 1))
          pol_reg = LinearRegression()
          model.append('PolR')
          warnings.filterwarnings('ignore')
          print('Metric: ',self.data_columns[i])
          print('Level: ',self.level)
          print('Model: ',j)
          print()
          print("**************************************************************")
          print()
          print('At i: ',i,' x = ',self.X_np[:,i])
          print()

          for train_index, test_index in kf.split(temp_data):
            cnt = cnt+1
            #print('Fold: ',cnt)
            X_train, X_test = temp_data[train_index], temp_data[test_index]
            y_train, y_test = self.Y[train_index], self.Y[test_index]

            pol_reg.fit(X_train,y_train)
            y_pred = pol_reg.predict(X_test)
            RMSE.append(np.sqrt(mean_squared_error(y_test,y_pred)))
            MAE.append(mean_absolute_error(y_test,y_pred))
          
          mean_RMSE.append(statistics.mean(RMSE))
          stdv_RMSE.append(statistics.pstdev(RMSE))
          mean_MAE.append(statistics.mean(MAE))
          stdv_MAE.append(statistics.pstdev(MAE))

        else:
          
          warnings.filterwarnings('ignore')
          #['Polynomial Regression','MLPRegressor','BayesianRidge','SGD Regressor','XGB Regressor','Decision Tree Regressor','ElasticNet','SVM Regressor',]
          if j == 'MLPR':
            reg = MLPRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          #if j == 'Logistic Regression':
            #reg = LogisticRegression()
            #print('Metric: ',self.data_columns[i])
            #print('Level: ',self.level)
            #print('Model: ',j)

          if j == 'BR':
            reg = BayesianRidge()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'SGDR':
            reg = SGDRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'XGBR':
            reg = XGBRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'DTR':
            reg = DecisionTreeRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'EN':
            reg = ElasticNet()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'SVR':
            reg = SVR()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)
          
          print('At i: ',i,' x = ',self.X_np[:,i])
          print()
          temp_data = self.X_np[:,i].reshape(-1, 1)
          target_scaler = MinMaxScaler()
          target_scaler.fit(temp_data)
          temp_data = target_scaler.transform(temp_data)
          level.append(self.level)
          metric.append(self.data_columns[i])

          cnt=0
          RMSE=[]
          MAE=[]
          for train_index, test_index in kf.split(temp_data):
            cnt = cnt+1
            #print('Fold: ',cnt)
            X_train, X_test = temp_data[train_index], temp_data[test_index]
            y_train, y_test = self.Y[train_index], self.Y[test_index]

            reg.fit(X_train,y_train)
            y_pred = reg.predict(X_test)
            RMSE.append(np.sqrt(mean_squared_error(y_test,y_pred)))
            MAE.append(mean_absolute_error(y_test,y_pred))

          mean_RMSE.append(statistics.mean(RMSE))
          stdv_RMSE.append(statistics.pstdev(RMSE))
          mean_MAE.append(statistics.mean(MAE))
          stdv_MAE.append(statistics.pstdev(MAE))
      

       

      #break
    metric = pd.DataFrame(metric,columns=['Metric'])
    level = pd.DataFrame(level,columns=['Level'])
    model = pd.DataFrame(model,columns=['Model'])
    print()

    mean_RMSE = pd.DataFrame(mean_RMSE,columns=['RMSE'])
    stdv_RMSE = pd.DataFrame(stdv_RMSE,columns=['STD RMSE'])
    mean_MAE = pd.DataFrame(mean_MAE,columns=['MAE'])
    stdv_MAE = pd.DataFrame(stdv_MAE,columns=['STD MAE'])

    res_df = pd.concat([metric,level,model,mean_RMSE,stdv_RMSE,mean_MAE,stdv_MAE],axis=1)

    return res_df

In [47]:
warnings.filterwarnings('ignore')
rtm=ReuseTradML(level='package')
res=rtm.apply_ml()
res

Metric:  AD
Level:  package
Model:  PolR

**************************************************************

At i:  0  x =  [0.       0.       0.       ... 0.540636 0.7      0.458716]

Metric:  AD
Level:  package
Model:  MLPR

**************************************************************

At i:  0  x =  [0.       0.       0.       ... 0.540636 0.7      0.458716]

Metric:  AD
Level:  package
Model:  BR

**************************************************************

At i:  0  x =  [0.       0.       0.       ... 0.540636 0.7      0.458716]

Metric:  AD
Level:  package
Model:  SGDR

**************************************************************

At i:  0  x =  [0.       0.       0.       ... 0.540636 0.7      0.458716]

Metric:  AD
Level:  package
Model:  XGBR

**************************************************************

At i:  0  x =  [0.       0.       0.       ... 0.540636 0.7      0.458716]

[01:08:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprec

,Metric,Level,Model,RMSE,STD RMSE,MAE,STD MAE
0,AD,package,PolR,0.022943,0.019580,0.005151,0.001528
1,AD,package,MLPR,0.021867,0.021102,0.006075,0.001796
2,AD,package,BR,0.023153,0.019264,0.005048,0.001293
3,AD,package,SGDR,0.022926,0.019544,0.005167,0.001474
4,AD,package,XGBR,0.025248,0.024750,0.005061,0.002529
...,...,...,...,...,...,...,...
139,NCL,package,SGDR,0.022445,0.020192,0.005234,0.001671
140,NCL,package,XGBR,0.021661,0.021508,0.005251,0.001887
141,NCL,package,DTR,0.023021,0.020256,0.005315,0.001712
142,NCL,package,EN,0.021659,0.021026,0.005241,0.001809


In [48]:
res.to_excel('/content/ReusabilityRate_ML/Results/packageRes_TraditionalModels_refined.xlsx')